# Chapter 2. Molecular Mechanics

## 2.1. Molecular Geometry

In this notebook, we will explore the fundamental concepts of molecular geometry, which play a crucial role in molecular mechanics simulations. Understanding these geometry terms is essential for accurately representing and analyzing molecular structures.

### 2.1.1. Bond Length

Bond length refers to the distance between the nuclei of two atoms that are bonded together in a molecule. It is a fundamental parameter in molecular geometry and affects the chemical properties of the molecule.

We can calculate the bond length between two atoms using their Cartesian coordinates:

$$\text{Bond Length} = \sqrt{{(x_1 - x_2)^2 + (y_1 - y_2)^2 + (z_1 - z_2)^2}}$$

We will explore an example with ethane molecule using RDKit:

In [1]:
# XYZ string for ethane (C2H6)
ethane_xyz = """8
Ethane
C      -0.765806   -0.000316   0.000000
C      0.765806    0.000316    0.000000
H      -1.165351   1.040005    -0.000000
H      -1.164581   -0.520796   0.901055
H      -1.164581   -0.520796   -0.901055
H      1.165351    -1.040005   0.000000
H      1.164581    0.520796    0.901055
H      1.164581    0.520796    -0.901055
"""

Here, using the coordinates of the 2 carbon atoms, the C-C bond length is:

$$\text{C-C Bond Length} = \sqrt{{(-0.765806 - 0.765806)^2 + (-0.000316 - 0.000316)^2 + (0.000000 - 0.000000)^2}} = 1.536112\text{ (Å)}$$

However, if you want to calculate any bond length, you should use RDKit:

In [2]:
# Import modules
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from utils import MolFromXYZWithSMILES, View3DModel # These are functions from previous sections

In [3]:
# Create an RDKit molecule from the XYZ string
smiles = 'CC'
ethane_mol = MolFromXYZWithSMILES(ethane_xyz, smiles)

# Visualize the molecule with Py3DMol
View3DModel(ethane_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
# Extract the coordinates of the atoms
carbon1 = ethane_mol.GetConformer(0).GetAtomPosition(0)
carbon2 = ethane_mol.GetConformer(0).GetAtomPosition(1)
hydrogen1 = ethane_mol.GetConformer(0).GetAtomPosition(2)

# Calculate the bond lengths
C_C_bond_length = (sum((carbon1[i] - carbon2[i]) ** 2 for i in range(3))) ** 0.5
C_H_bond_length = (sum((carbon1[i] - hydrogen1[i]) ** 2 for i in range(3))) ** 0.5

# Display the bond length
print(f"The bond length between C1 and C2 in ethane is {C_C_bond_length:.4f} Å.")
print(f"The bond length between C1 and H1 in ethane is {C_H_bond_length:.4f} Å.")

The bond length between C1 and C2 in ethane is 1.5316 Å.
The bond length between C1 and H1 in ethane is 1.1144 Å.


Alternatively, we can get the bond length using rdkit.Chem.rdMolTransforms module:

In [5]:
# Calculate the bond lengths
C_C_bond_length = Chem.rdMolTransforms.GetBondLength(ethane_mol.GetConformer(0), 0, 1)
C_H_bond_length = Chem.rdMolTransforms.GetBondLength(ethane_mol.GetConformer(0), 0, 2)

# Display the bond length
print(f"The bond length between C1 and C2 in ethane is {C_C_bond_length:.4f} Å.")
print(f"The bond length between C1 and H1 in ethane is {C_H_bond_length:.4f} Å.")

The bond length between C1 and C2 in ethane is 1.5316 Å.
The bond length between C1 and H1 in ethane is 1.1144 Å.


Using RDKit, we can also manipulate the bond length of a molecule. For example:

In [6]:
# Double the C-C bond length
new_C_C_bond_length = C_C_bond_length * 2

# Set the new C-C bond length
Chem.rdMolTransforms.SetBondLength(ethane_mol.GetConformer(0), 0, 1, new_C_C_bond_length)

# Visualize the molecule with Py3DMol
View3DModel(ethane_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
# To confirm, we can print out the C-C bond length again
print(Chem.rdMolTransforms.GetBondLength(ethane_mol.GetConformer(0), 0, 1))

3.0632242607866633


### 2.1.2. Bond Angle

Bond angle refers to the angle between two bonds that share a common atom. It is crucial for determining the molecular shape and properties.

The equation to calculate bond angle equation is as follows:

$$\text{Bond Angle} = \arccos\left(\frac{{\mathbf{A} \cdot \mathbf{B}}}{{\lVert \mathbf{A} \rVert \cdot \lVert \mathbf{B} \rVert}}\right)$$

Where **A** and **B** represents the vectors corresponding to the bonds that form the angle.

We will calculate the H-C-C angle in ethane using numpy:

In [8]:
# Create the molecule again
smiles = 'CC'
ethane_mol = MolFromXYZWithSMILES(ethane_xyz, smiles)

# Extract the coordinates of the atoms
carbon1 = ethane_mol.GetConformer(0).GetAtomPosition(0)
carbon2 = ethane_mol.GetConformer(0).GetAtomPosition(1)
hydrogen1 = ethane_mol.GetConformer(0).GetAtomPosition(2)

# Calculate vectors
vector1 = np.array(hydrogen1) - np.array(carbon1)
vector2 = np.array(carbon2) - np.array(carbon1)

# Calculate bond angle in degrees
H_C_C_bond_angle_rad = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
H_C_C_bond_angle_deg = np.degrees(H_C_C_bond_angle_rad)

# Display the bond angle
print(f"The H-C-C bond angle in ethane is {H_C_C_bond_angle_deg:.2f} degrees.")

The H-C-C bond angle in ethane is 110.99 degrees.


Alternatively, we can get the bond angle using rdkit.Chem.rdMolTransforms module:

In [9]:
# Calculate the bond angle
H_C_C_bond_angle_deg = Chem.rdMolTransforms.GetAngleDeg(ethane_mol.GetConformer(0), 2, 0, 1)

# Display the bond angle
print(f"The H-C-C bond angle in ethane is {H_C_C_bond_angle_deg:.2f} degrees.")

The H-C-C bond angle in ethane is 110.99 degrees.


Using RDKit, we can also manipulate the bond angle of a molecule. For example:

In [10]:
# Set the H-C-C bond angle to a new value
new_H_C_C_bond_angle_deg = 45

# Set the new H-C-C bond angle
Chem.rdMolTransforms.SetAngleDeg(ethane_mol.GetConformer(0), 2, 0, 1, new_H_C_C_bond_angle_deg)

# Visualize the molecule with Py3DMol
View3DModel(ethane_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
# To confirm, we can print out the H-C-C bond angle again
print(Chem.rdMolTransforms.GetAngleDeg(ethane_mol.GetConformer(0), 2, 0, 1))

44.99999999999998


### 2.1.3. Torsion Angle

Torsion angle, also known as dihedral angle, refers to the angle between two planes in a molecule. It's essential for understanding the conformational flexibility of molecules.

![Torsion angle](./images/dihedral_angle.png)

![Torsion angle in molecule](./images/dihedral_angle_molecule.png)

We can calculate torsion angle using the following equation:

$$\text{Dihedral Angle} = \arctan2\left( \mathbf{B} \cdot ((\mathbf{A} \times \mathbf{B}) \times(\mathbf{B} \times \mathbf{C})), | \mathbf{B} | \times (\mathbf{A} \times \mathbf{B}) \times(\mathbf{B} \cdot \mathbf{C}) \right)$$

Where:

- **A** represents the vector of first bond.
- **B** represents the vector of second bond.
- **C** represents the vector of third bond.

We will calculate the H-C-C-H torsion angle in ethane using numpy:

In [12]:
# Create the molecule again
smiles = 'CC'
ethane_mol = MolFromXYZWithSMILES(ethane_xyz, smiles)

# Extract the coordinates of the atoms
carbon1 = ethane_mol.GetConformer(0).GetAtomPosition(0)
carbon2 = ethane_mol.GetConformer(0).GetAtomPosition(1)
hydrogen1 = ethane_mol.GetConformer(0).GetAtomPosition(2)
hydrogen2 = ethane_mol.GetConformer(0).GetAtomPosition(6)

# Calculate vectors
vector1 = carbon1 - hydrogen1
vector2 = carbon2 - carbon1
vector3 = hydrogen2 - carbon2

# Calculate torsion angle in degrees
cross_product1 = np.cross(vector1, vector2)
cross_product2 = np.cross(vector2, vector3)
H_C_C_H_torsion_angle_rad = np.arctan2(np.dot(vector2, np.cross(cross_product1, cross_product2)), np.linalg.norm(vector2) * np.dot(cross_product1, cross_product2))
H_C_C_H_torsion_angle_deg = np.degrees(H_C_C_H_torsion_angle_rad)

# Display the torsion angle
print(f"The H-C-C-H torsion angle in ethane is {H_C_C_H_torsion_angle_deg:.2f} degrees.")

The H-C-C-H torsion angle in ethane is 60.00 degrees.


Alternatively, we can get the torsion angle using rdkit.Chem.rdMolTransforms module:

In [13]:
# Calculate the torsion angle
H_C_C_H_torsion_angle_deg = Chem.rdMolTransforms.GetDihedralDeg(ethane_mol.GetConformer(0), 2, 0, 1, 6)

# Display the torsion angle
print(f"The H-C-C bond angle in ethane is {H_C_C_H_torsion_angle_deg:.2f} degrees.")

The H-C-C bond angle in ethane is 60.00 degrees.


Using RDKit, we can also manipulate the torsion angle of a molecule. For example:

In [14]:
# Set the H-C-C-H torsion angle to a new value
new_H_C_C_H_torsion_angle_deg = 45

# Set the new H-C-C-H torsion angle
Chem.rdMolTransforms.SetDihedralDeg(ethane_mol.GetConformer(0), 2, 0, 1, 6, new_H_C_C_H_torsion_angle_deg)

# Visualize the molecule with Py3DMol
View3DModel(ethane_mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
# To confirm, we can print out the H-C-C-H torsion angle again
print(Chem.rdMolTransforms.GetDihedralDeg(ethane_mol.GetConformer(0), 2, 0, 1, 6))

44.99999999999998
